# Genomics Application for Spectral Clustering: A Gentle Introduction 
This notebook provides the executable code and visualization steps for the concepts detailed in the corresponding section of the paper, "A gentle introduction to spectral clustering."

The Data used in this notebook was produced and published by Bajtai and colleagues, doi: 10.1186/s12943-025-02310-0

In the present script, we are using bulk RNA-seq data, preprocessed as described in the original publication. The similarity matrix was calculated using distance correlation, and the graph Laplacian using a random-walk normalization. 

## Load data and libraries

In [1]:
load("data/bulk_RNAseq_data.RData")

In [2]:
library(extrafont)
font_import() 
loadfonts(device = "pdf")
font_family <- "DejaVu Sans Light"

Warning message:
“package ‘extrafont’ was built under R version 4.4.3”
Registering fonts with R



Importing fonts may take a few minutes, depending on the number of fonts and the speed of the system.
Continue? [y/n]  y


Scanning ttf files in /usr/share/fonts/, /usr/local/share/fonts/ ...

Extracting .afm files from .ttf files...

/usr/share/fonts/truetype/dejavu/DejaVuMathTeXGyre.ttf
 : DejaVuMathTeXGyre-Regular already registered in fonts database. Skipping.

/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf
 : DejaVuSans-Bold already registered in fonts database. Skipping.

/usr/share/fonts/truetype/dejavu/DejaVuSans-BoldOblique.ttf
 : DejaVuSans-BoldOblique already registered in fonts database. Skipping.

/usr/share/fonts/truetype/dejavu/DejaVuSans-ExtraLight.ttf
 : DejaVuSans-ExtraLight already registered in fonts database. Skipping.

/usr/share/fonts/truetype/dejavu/DejaVuSans-Oblique.ttf
 : DejaVuSans-Oblique already registered in fonts database. Skipping.

/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf
 : DejaVuSans already registered in fonts database. Skipping.

/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed-Bold.ttf
 : DejaVuSansCondensed-Bold already registered in fonts database. 

In [3]:
set.seed(12345)

## First look at eigenvalues and eigenvectors

In [4]:
# Plot eigenvalues, a.k.a. the spectrum
pdf(file = "application_spectrum.pdf", width = 6, height = 6)
par(family = font_family)
plot(evals, ylim = c(0.85, 1), pch = 1.5,
     main = "Non-trivial eigen values of the Graph Laplacian", ylab = "Eigen values", xlab = "Index")
dev.off()

pdf 
  2

In [5]:
# Get the number of clusters k with the elbow method
df <- data.frame(x = 1:length(evals), 
                 y = evals)
k <- pathviewr::find_curve_elbow(df[-1 , ]) # discard the trivial eigenvalue
k

[1] 17

In [6]:
# The linegraph of eigenfunctions: for large data, too obscure
order <- order(evecs$V1)

pdf("images/application_linegraph.pdf", width = 6, height = 6)
par(family = font_family)
plot((evecs$V2)[order], col = scales::alpha("chartreuse3", .5), 
     ylim = c(-.05, .05), type = "l",
     main = "Eigenfunctions 2-3", 
     ylab = "Values", xlab = "Nodes")
lines(evecs$V3[order], col = scales::alpha("steelblue", .5))
dev.off()

pdf 
  2

## PCA overview of the data and the eigenvalues and eigenvectors

In [7]:
# Get the first two principal components
loadings <- pca$rotation[, seq_len(2)] # PCA object was obtained by DESeq2 with default thresholding and unsing the 500 most variable features

In [8]:
# Set up colors for different conditions
metadata$condition_color <- ifelse(metadata$condition == "ctr", "darkblue",
                                  ifelse(metadata$condition == "tis", "aquamarine3",
                                        ifelse(metadata$condition == "repop", "darkgoldenrod1",
                                            "grey")))

metadata$cell_line_color <- ifelse(metadata$cell_line == "MCF7", "steelblue",
                                  ifelse(metadata$cell_line == "T47D", "coral2",
                                        ifelse(metadata$cell_line == "HFF", "darkmagenta",
                                            ifelse(metadata$cell_line == "Hs578T", "darkolivegreen3",
                                                ifelse(metadata$cell_line == "MDA-MB-231", "burlywood2",
                                                    "grey")))))

metadata$mes_vs_lum_color <- ifelse(metadata$cell_line == "MCF7", "steelblue",
                                  ifelse(metadata$cell_line == "T47D", "steelblue",
                                        "coral2"))

In [9]:
# Set up figure legend
condition_labels <- c("ctr", "tis", "repop", "Other")
condition_colors <- c("darkblue", "aquamarine3", "darkgoldenrod1", "grey")

cell_line_labels <- c("MCF7", "T47D", "HFF", "Hs578T", "MDA-MB-231", "Other")
cell_line_colors <- c("steelblue", "coral2", "darkmagenta", "darkolivegreen3", "burlywood2", "grey")

mes_vs_lum_labels <- c("Luminal", "Mesenchymal")
mes_vs_lum_colors <- c("steelblue", "coral2")

In [19]:
# Plot PCA plot of conditions and cancer subtypes to see any natural clusters
pdf("images/application_pca_treatment.pdf", width = 6, height = 6)
par(family = font_family)
plot(loadings, pch = 21, cex = 2, bg = metadata$condition_color, col = "black",  main = "Treatment condition" )
legend("topright",
       legend = condition_labels,
       pt.bg = condition_colors,
       col = "black",
       pch = 21, 
       pt.cex = 1.5, 
       title = "Condition"
)
dev.off()

pdf("images/application_pca_subtype.pdf", width = 6, height = 6)
par(family = font_family)
plot(loadings, pch = 21, cex = 2, bg = metadata$cell_line_color, col = "black", main = "Molecular subtype" )
legend("topright",
       legend = cell_line_labels,
       pt.bg = cell_line_colors,
       col = "black",
       pch = 21, 
       pt.cex = 1.5, 
       title = "Subtype"
)
dev.off()

pdf("images/application_pca_subtype_lum_vs_mesenchymal.pdf", width = 6, height = 6)
par(family = font_family)
plot(loadings, pch = 21, cex = 2, bg = metadata$mes_vs_lum_color, col = "black", main = "Luminal vs Mesenchymal Subtype" )
legend("topright",
       legend = mes_vs_lum_labels,
       pt.bg = mes_vs_lum_colors,
       col = "black",
       pch = 21, 
       pt.cex = 1.5, 
       title = "Subtype"
)
dev.off()

pdf 
  2

pdf 
  2

pdf 
  2

In [11]:
# Plot PCA plots colored by eigenvectors to highlight what natural clusters they might mark

ncolors = 20
colfunc <- colorRampPalette(c("blue3", "darkturquoise", "gold1"))(ncolors)

pdf("images/application_pca_evec2.pdf", width = 6, height = 6)
par(family = font_family)
plot(loadings, pch = 21, cex = 2, bg = colfunc[cut(metadata$evec2, ncolors)], col = "black", main = "Second eigenvector" )
dev.off()

pdf("images/application_pca_evec3.pdf", width = 6, height = 6)
par(family = font_family)
plot(loadings, pch = 21, cex = 2, bg = colfunc[cut(metadata$evec3, ncolors)], col = "black", main = "Third eigenvector" )
dev.off()

pdf("images/application_pca_evec4.pdf", width = 6, height = 6)
par(family = font_family)
plot(loadings, pch = 21, cex = 2, bg = colfunc[cut(metadata$evec4, ncolors)], col = "black", main = "Fourth eigenvector" )
dev.off()

colfunc <- colorRampPalette(c("maroon2", "blue3", "darkturquoise", "gold1"))(ncolors)
pdf("images/application_pca_evec5.pdf", width = 6, height = 6)
par(family = font_family)
plot(loadings, pch = 21, cex = 2, bg = colfunc[cut(metadata$evec5, ncolors)], col = "black", main = "Fifth eigenvector" )
dev.off()

pdf 
  2

pdf 
  2

pdf 
  2

pdf 
  2

## Heatmap of spectral clustering results and the eigenspace

In [12]:
# Function for plotting the heatmap of eigenvectors

# This function performs K-means clustering on selected eigenvectors from a spectral analysis.
# It takes an 'eigen_analysis' object, the desired number of clusters, and an option
# to display a heatmap of the clustered eigenvectors.
# num_clusters should be determined by the spectrum of the laplacian (e.g. number of zero eigenvalues, or index of the elbow)
# The K-means result (including cluster assignments) is returned.
eigen_kmeans <- function(eigen_analysis, num_clusters, is_heatmap = T, title = "", 
                         pdf_filename = NULL, pdf_width = 7, pdf_height = 7){

    # 1. K-MEANS CLUSTERING SETUP
    set.seed(123)
    order <- order(abs(eigen_analysis$eigenvalues), decreasing = F)
    my_matrix <- eigen_analysis$eigenvectors[,order][ , 2:(num_clusters-1)] 
    rownames(my_matrix) <- colnames(eigen_analysis$laplacian_matrix)
    kmeans_result <- stats::kmeans(my_matrix, centers = num_clusters, nstart = 25, iter.max = 1000)

    # 2. HEATMAP GENERATION AND PDF SAVING
    if(is_heatmap){
        
        # Get and order the matrix by cluster assignment
        row_clusters <- kmeans_result$cluster
        ordered_row_indices <- order(row_clusters)
        ordered_matrix <- my_matrix[ordered_row_indices, ]
        row_clusters <- row_clusters[ordered_row_indices] # Ensure clusters vector is also ordered

        # Create the row annotation data frame for displaying cluster identity
        row_annotation_df <- data.frame(
            Cluster = factor(row_clusters) # Cluster IDs are converted to factors for discrete colors
        )
        rownames(row_annotation_df) <- rownames(ordered_matrix)

        # column annotation
        col_names <- colnames(ordered_matrix)
        italic_col_labels <- lapply(col_names, function(x) bquote(italic(.(x))))
                                    
        # Determine output: if filename is NULL, plot to screen (default R device)
        output_params <- list(filename = pdf_filename, width = pdf_width, height = pdf_height)
        if (is.null(pdf_filename)) {
            # Use options for screen device size if not saving to file
            options(repr.plot.width = 7, repr.plot.height = 7)
            output_params <- list() # Clear file output parameters
        }

        # Generate the heatmap
        pheatmap::pheatmap(ordered_matrix,
                           filename = output_params$filename, 
                           width = output_params$width,
                           height = output_params$height,
                           cluster_rows = FALSE, 
                           cluster_cols = FALSE, 
                           show_rownames = FALSE, 
                           show_colnames = TRUE, 
                           fontfamily = font_family,
                           annotation_row = row_annotation_df, 
                           labels_col = as.expression(italic_col_labels),
                           color = colorRampPalette(c("darkslateblue", "white", "coral3"))(50), 
                           main = title ,
                           gaps_row = cumsum(table(row_clusters))[-num_clusters] 
        )
    }

    return(kmeans_result)
}

In [15]:
# Make a list
egien_analysis_list <- list(eigenvalues = evals, eigenvectors = evecs, laplacian_matrix = laplacian)

In [18]:
# Plot heatmap
kmeans_result <- eigen_kmeans(egien_analysis_list, num_clusters = k+1, is_heatmap = T, 
                              pdf_filename = "images/application_eigen_heatmap.pdf", pdf_width = 7, pdf_height = 7,
                              title = "Heatmap of eigenvectors 2:16 as columns, \nrows are clustered using kmeans with k=17")

In [17]:
# Display the gene ID in cluster 8
# paste(names(kmeans_result$cluster)[kmeans_result$cluster == 8], collapse = " ") 

[1] "ENSG00000159423 ENSG00000142675 ENSG00000126698 ENSG00000163867 ENSG00000130940 ENSG00000126067 ENSG00000163866 ENSG00000224609 ENSG00000116212 ENSG00000188786 ENSG00000117174 ENSG00000117480 ENSG00000116299 ENSG00000137962 ENSG00000069702 ENSG00000117525 ENSG00000162695 ENSG00000117262 ENSG00000137947 ENSG00000184371 ENSG00000162694 ENSG00000143178 ENSG00000143355 ENSG00000151414 ENSG00000133069 ENSG00000118454 ENSG00000243364 ENSG00000143590 ENSG00000169241 ENSG00000160049 ENSG00000133019 ENSG00000163472 ENSG00000162734 ENSG00000188092 ENSG00000225080 ENSG00000070785 ENSG00000126088 ENSG00000171488 ENSG00000117751 ENSG00000282057 ENSG00000142871 ENSG00000186063 ENSG00000143067 ENSG00000143578 ENSG00000162642 ENSG00000223653 ENSG00000077254 ENSG00000143344 ENSG00000117569 ENSG00000215915 ENSG00000159214 ENSG00000117505 ENSG00000142669 ENSG00000162517 ENSG00000183682 ENSG00000117385 ENSG00000084652 ENSG00000253304 ENSG00000198160 ENSG00000154222 ENSG00000197147 ENSG00000131236 ENSG00000142920 ENSG00000117226 ENSG00000117228 ENSG00000153904 ENSG00000117020 ENSG00000163382 ENSG00000143303 ENSG00000123091 ENSG00000085831 ENSG00000122477 ENSG00000137996 ENSG00000025800 ENSG00000154511 ENSG00000162623 ENSG00000143412 ENSG00000163141 ENSG00000143443 ENSG00000197622 ENSG00000143479 ENSG00000116863 ENSG00000116871 ENSG00000225313 ENSG00000197956 ENSG00000160678 ENSG00000214193 ENSG00000181817 ENSG00000067208 ENSG00000163344 ENSG00000160685 ENSG00000163354 ENSG00000271601 ENSG00000185483 ENSG00000169641 ENSG00000143409 ENSG00000142751 ENSG00000132849 ENSG00000011007 ENSG00000117114 ENSG00000186364 ENSG00000116489 ENSG00000127603 ENSG00000117153 ENSG00000272084 ENSG00000162704 ENSG00000116698 ENSG00000133056 ENSG00000230638 ENSG00000183291 ENSG00000180488 ENSG00000272574 ENSG00000143126 ENSG00000117143 ENSG00000163565 ENSG00000117595 ENSG00000203667 ENSG00000130762 ENSG00000174021 ENSG00000116717 ENSG00000172380 ENSG00000116729 ENSG00000179456 ENSG00000162368 ENSG00000126070 ENSG00000232284 ENSG00000271576 ENSG00000287839 ENSG00000203706 ENSG00000143373 ENSG00000143375 ENSG00000219201 ENSG00000162614 ENSG00000230021 ENSG00000143702 ENSG00000122417 ENSG00000186283 ENSG00000181773 ENSG00000169991 ENSG00000116260 ENSG00000097033 ENSG00000290705 ENSG00000169504 ENSG00000211451 ENSG00000134686 ENSG00000196208 ENSG00000171055 ENSG00000186854 ENSG00000135631 ENSG00000115091 ENSG00000238133 ENSG00000170485 ENSG00000144120 ENSG00000081320 ENSG00000186522 ENSG00000272211 ENSG00000172965 ENSG00000170035 ENSG00000138434 ENSG00000230076 ENSG00000144452 ENSG00000065802 ENSG00000118242 ENSG00000196141 ENSG00000138386 ENSG00000119878 ENSG00000115825 ENSG00000163171 ENSG00000132326 ENSG00000156973 ENSG00000144036 ENSG00000134324 ENSG00000286207 ENSG00000115226 ENSG00000235655 ENSG00000169599 ENSG00000204634 ENSG00000118257 ENSG00000273045 ENSG00000114988 ENSG00000158158 ENSG00000018510 ENSG00000125637 ENSG00000177483 ENSG00000173559 ENSG00000115977 ENSG00000115963 ENSG00000124380 ENSG00000170500 ENSG00000072163 ENSG00000243943 ENSG00000176714 ENSG00000135932 ENSG00000125629 ENSG00000115355 ENSG00000124839 ENSG00000143994 ENSG00000183671 ENSG00000138356 ENSG00000115295 ENSG00000119771 ENSG00000079150 ENSG00000138101 ENSG00000231908 ENSG00000155066 ENSG00000143882 ENSG00000163249 ENSG00000179833 ENSG00000188687 ENSG00000003400 ENSG00000134318 ENSG00000151689 ENSG00000243147 ENSG00000228925 ENSG00000180398 ENSG00000143933 ENSG00000115310 ENSG00000169764 ENSG00000119185 ENSG00000119203 ENSG00000134308 ENSG00000085449 ENSG00000224165 ENSG00000222041 ENSG00000226124 ENSG00000272156 ENSG00000136717 ENSG00000153823 ENSG00000204923 ENSG00000173705 ENSG00000131378 ENSG00000114353 ENSG00000206538 ENSG00000144744 ENSG00000249992 ENSG00000176095 ENSG00000241316 ENSG00000163513 ENSG00000170142 ENSG00000001617 ENSG00000144824 ENSG00000240891 ENSG00000031081 ENSG00000168917 ENSG00000058063 ENSG00000237978 ENSG00000152492 ENSG00000114315 ENSG00000213123 ENSG0000028